In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import os 
import glob
import numpy as np
import matplotlib.pyplot as plt 
import cv2
import multiprocessing
import random
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [ ]:
model = tf.keras.models.load_model("/content/gdrive/MyDrive/resnet18_16_3")

In [ ]:
cpu = multiprocessing.cpu_count() 
print("cpu_count :", cpu)

# 파일에 들어있는 이미지 갯수 확인하기 
os.chdir('/content/gdrive/MyDrive/projectimg')
file_name = os.listdir()
cwd = os.getcwd()

img_path = [] 
for i in range(len(file_name)):
  
  path = os.path.join(cwd, file_name[i])
  print(file_name[i],":",len(os.listdir(path)))
  # 파일에 들어있는 이미지의 경로를 리스트 형태로 구현 
  img_path.append(list(glob.glob(os.path.join(path,"*"))))

cpu_count : 40
Pet : 5000
Person : 5526
Kakao-talk : 5432
Game : 5000
Food : 5753
Unknown : 17321


새로운 데이터를 추가하고 기존데이터를 랜덤으로 골라서 5000개를 맞추는코드.

In [ ]:

# tmp_person = img_path[1][504:1004]
# tmp_kakao = img_path[2][568:999]
# tmp_food = img_path[4][231:999]
# tmp_person2 = img_path[1][:503]
# tmp_person3 = img_path[1][1005:5496]
# tmp_kakao2 = img_path[2][:567]
# print(len(tmp_kakao2))
# tmp_kakao3 = img_path[2][1000:5431]
# tmp_food2 = img_path[4][:230]
# tmp_food3 = img_path[4][1000:5768]
# img_path[1] = tmp_person2
# img_path[2] = tmp_kakao2
# img_path[4] = tmp_food2

# for imgs in tmp_person3:
#   img_path[1].append(imgs)
  
# for imgs in tmp_kakao3:
#   img_path[2].append(imgs)
  
# for imgs in tmp_food3:
#   img_path[4].append(imgs)

새로운데이터와 기존데이터를 전부 섞어서 5000개를 맞추는코드.

In [ ]:
random.shuffle(img_path[1])
random.shuffle(img_path[2])
random.shuffle(img_path[4])
img_path[1] = img_path[1][0:5000]
img_path[2] = img_path[2][0:5000]
img_path[4] = img_path[4][0:5000]
random.shuffle(img_path[3])
img_path[3] = img_path[3][:5000]

# for imgs in tmp_person:
#   img_path[1].append(imgs)
# for imgs in tmp_kakao:
#   img_path[2].append(imgs)
# for imgs in tmp_food:
#   img_path[4].append(imgs)


In [ ]:
# unknown에서 5000개 뽑는 과정
count = 0
random.shuffle(img_path[5])
# tmp = img_path[5]
# tmp = tmp[:5000]

# for i in tmp:
#   img = cv2.imread(i)
#   if img is None:
#       print("Img is None:",i)
#   count = count + 1
#   if count % 500 == 0:
#     print(count,"th")
img_path[5] = img_path[5][:5000]

In [ ]:
#데이터셋 스플릿
def split(List):
  # shuffle
  random.shuffle(List)  

  # train data : test data = 8 : 2
  split_ratio = int(len(List) * 0.2)
  test_set = List[ :split_ratio]
  train_set = List[split_ratio: ]
  return train_set, test_set

train, test = [[],[],[],[],[],[]], [[],[],[],[],[],[]]
train_len, test_len = 0, 0

# img_path[0] ~ img_path[5]: 각 파일에 있는 이미지경로 
for i in range(len(img_path)):  
   train[i], test[i] = split(img_path[i])
   
   #print(file_name[i],"- train_set:",len(train[i]),", test_set",len(test[i]))
   train_len += len(train[i])
   test_len += len(test[i])
   print(file_name[i],"- train_set:",len(train[i]),", test_set",len(test[i]))


print("Num of train:",train_len,", Num of test:",test_len)

Pet - train_set: 4000 , test_set 1000
Person - train_set: 4000 , test_set 1000
Kakao-talk - train_set: 4000 , test_set 1000
Game - train_set: 4000 , test_set 1000
Food - train_set: 4000 , test_set 1000
Unknown - train_set: 4000 , test_set 1000
Num of train: 24000 , Num of test: 6000


In [ ]:
#모델 구현
#총 18개의 컨볼루션 래여, abced 5개의 블럭이 존재한다.
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, ZeroPadding2D, MaxPool2D,Add,Flatten,Dense,GlobalAveragePooling2D,Input
def convA(x):
  
  x = Conv2D( 64, (7,7),strides = (2,2),padding = 'same')(x)
  x = BatchNormalization()(x)
  x=  Activation('relu')(x)
  x = MaxPool2D((3,3),(1,1),padding ='same')(x)

  return x

def convB(x): 
  shortcut = x
  x = Conv2D(64,(3,3),strides = (2,2),padding = 'same')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(64,(3,3),strides = (1,1),padding = 'same')(x)
  shortcut = Conv2D(64,(1,1),strides = (2,2), padding = 'valid')(shortcut)
  shortcut = BatchNormalization()(shortcut) 
  x = Add()([x,shortcut])
  x = Activation('relu')(x)
  
  shortcut = x

  x = Conv2D(64,(3,3),strides = (1,1),padding ='same')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(64,(3,3),strides = (1,1),padding='same')(x)
  x = BatchNormalization()(x)
  x = Add()([x,shortcut])
  x = Activation('relu')(x)

  return x

def convC(x):
  shortcut = x
  x = Conv2D(128,(3,3),strides = (2,2),padding='same')(x) 
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(128,(3,3),strides = (1,1),padding='same')(x)
  shortcut = Conv2D(128,(1,1),strides = (2,2), padding = 'valid')(shortcut)
  shortcut = BatchNormalization()(shortcut)
  x = BatchNormalization()(x)
  x = Add()([x,shortcut])
  x = Activation('relu')(x)
  shortcut = x

  x = Conv2D(128,(3,3),strides = (1,1),padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(128,(3,3),strides = (1,1),padding='same')(x)
  x = BatchNormalization()(x)
  x = Add()([x,shortcut])
  x = Activation('relu')(x)
  return x

def convD(x):
  shortcut = x
  x = Conv2D( 256,(3,3),strides = (2,2),padding='same')(x) 
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(256,(3,3),strides = (1,1),padding='same')(x)
  shortcut = Conv2D(256,(1,1),strides = (2,2), padding = 'valid')(shortcut)
  shortcut = BatchNormalization()(shortcut)
  x = BatchNormalization()(x)
  x = Add()([x,shortcut])
  x = Activation('relu')(x)

  shortcut = x

  x = Conv2D(256,(3,3),strides = (1,1),padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x =Conv2D(256,(3,3),strides = (1,1),padding='same')(x)
  x = BatchNormalization()(x)
  x = Add()([x,shortcut])
  x = Activation('relu')(x)
  return x

def convE(x):
  shortcut = x
  x = Conv2D(512,(3,3),strides = (2,2),padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D( 512,(3,3),strides = (1,1),padding='same')(x)
  shortcut = Conv2D(512,(1,1),strides = (2,2), padding = 'valid')(shortcut)
  shortcut = BatchNormalization()(shortcut)
  x = BatchNormalization()(x)
  x = Add()([x,shortcut])
  x = Activation('relu')(x)

  shortcut = x

  x = Conv2D( 512,(3,3),strides = (1,1),padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D( 512,(3,3),strides = (1,1),padding='same')(x)
  x = BatchNormalization()(x)
  x = Add()([x,shortcut])
  x = Activation('relu')(x)
  return x

input = Input(shape=(224, 224, 3), dtype='float32', name='input')
x = convA(input)
x = convB(x)
x = convC(x)
x = convD(x)
x = convE(x)
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
output = Dense(6,activation= 'softmax')(x) #레이벨 카테고리가 6개
model = tf.keras.Model(input,output)

loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.summary()





Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['input[0][0]']                  
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
def label(List, what):
   batch_y=[]
   if what == 0: img_agu_num = 1  # train_data일때(데이터 증강 적용)
   else: img_agu_num = 1  # test_data일때, train_data의 unknown부분 
  
   for i in List:

     dir,_ = os.path.split(i)
         
     if dir == '/content/gdrive/.shortcut-targets-by-id/1Mcaf62r3wusYpkZ4Dptx9O4KN16V0-02/projectimg/Pet': batch_y += [0]*img_agu_num
     elif dir == '/content/gdrive/.shortcut-targets-by-id/1Mcaf62r3wusYpkZ4Dptx9O4KN16V0-02/projectimg/Person': batch_y += [1]*img_agu_num
     elif dir == '/content/gdrive/.shortcut-targets-by-id/1Mcaf62r3wusYpkZ4Dptx9O4KN16V0-02/projectimg/Kakao-talk': batch_y += [2]*img_agu_num
     elif dir == '/content/gdrive/.shortcut-targets-by-id/1Mcaf62r3wusYpkZ4Dptx9O4KN16V0-02/projectimg/Game': batch_y += [3]*img_agu_num
     elif dir == '/content/gdrive/.shortcut-targets-by-id/1Mcaf62r3wusYpkZ4Dptx9O4KN16V0-02/projectimg/Food': batch_y += [4]*img_agu_num
     elif dir == '/content/gdrive/.shortcut-targets-by-id/1Mcaf62r3wusYpkZ4Dptx9O4KN16V0-02/projectimg/Unknown' : batch_y += [5]*img_agu_num
    
   return batch_y

데이터 증강을 지금은 뺏지만 만약 데이터 증강을 이용하려면 밑에있는 img1~6의 주석을 풀고 레이벨링 함수에서 img_agu_num을 증강하는 종류의 갯수에 따라 올린다.

In [ ]:
def preprocess(List, what):
  batch_x = []
  for r in List:
    img = cv2.imread(r)
    try:
      img = cv2.resize(img, (224,224), interpolation=cv2.INTER_AREA)
    except:
      pass

    if img is None:
      print("Img is None:",r)
      pass

    elif what == 0:
      # img1 = rotation(img).astype(np.float32) / 255.
      # img3 = zoom(img).astype(np.float32) / 255.
      # img4 = brightness(img).astype(np.float32) / 255.
      # #img5 = blur(img).astype(np.float32) / 255.
      # img6 = flip(img).astype(np.float32) / 255.
      img0 = img.astype(np.float32) / 255.
      #batch_x += [img0,img1,img3,img4,img6]
      batch_x += [img0]
    
    else:
      img = img.astype(np.float32) / 255.
      batch_x += [img]
      
  return batch_x

In [ ]:
# 1. Rotation 
def rotation(img):
    h,w = img.shape[:2] 
    center = (int(w/2), int(h/2)) 
    move = cv2.getRotationMatrix2D(center, np.random.randint(-30,30), 1.0)
    img = cv2.warpAffine(img,move,(w,h))
    return img

# 2. Flip 
def flip(img):
    # -1:상하좌우, 0:상하, 1:좌우
    case = np.random.randint(-1,1+1) 
    img = cv2.flip(img, 1)
    return img

# 3. Zoom 
def zoom(img):
    h,w = img.shape[:2] 
    n = np.random.uniform(0.5, 1, 1)
    # 확대할 부분의 시작 x좌표:w_start, y좌표:h_start 
    # 확대할 부분의 높이:n*h, 너비:n*w 
    h_start = np.random.randint(0, h - int(n*h))
    w_start = np.random.randint(0, w - int(n*w))
     
    img = img[h_start : h_start+int(n*h), w_start : w_start+int(n*w), ]
    img = cv2.resize(img, (h, w), cv2.INTER_CUBIC)
    return img 

# 4. Brightness 
def brightness(img):
    case = np.random.randint(1,2+1)
    v = np.random.choice(range(30,50+10,10))
    init = np.ones(img.shape, dtype = 'uint8') 
    
    if case == 1:
      img = cv2.add(img, init * v)
    else: 
      img = cv2.subtract(img, init * v)
    return img

# 5. 1) GaussianBlur, 2) medianBlur 
def blur(img):
    case = np.random.randint(1,2+1)
    k_size = 3
    if case == 1: 
        img = cv2.GaussianBlur(img, (k_size, k_size), 0)
    else: 
        img = cv2.medianBlur(img, k_size)
    return img



In [ ]:
model = tf.keras.models.load_model("/content/gdrive/MyDrive/resnet18_16_4_1")

In [ ]:
for i in range(12):#한번에 총 12번의 학습. 하지만 데이터 증강을 했을경우엔 4~5번이 maximum 왜냐하면 코랩 연결타임이 24시간이다.
  # 첫번째 epoch 
  batch_size = 60
  slice_num, slice_num_test = 0, 0
  # 배치사이즈로 나눠서 train하는 과정, 한 배치에 60

  batch_num = round(train_len / batch_size)
  tmp_slice = int(batch_size/len(train))

  tmp_slice_test = int(((test_len / batch_num)*10)/len(test))
  result_loss, train_a, test_a = [], [], [] 

  for i in range(batch_num):


    # 0,1,2,3,4,5에서 각각 10개씩 pick, label함수를 통해서 라벨링 
    batch_x = []
    for j in range(len(train)):

      tmp = train[j]
      batch_x += tmp[0+slice_num : tmp_slice+slice_num]
    slice_num += tmp_slice 

    random.shuffle(batch_x) 

    batch_y = np.array(label(batch_x,0))
    batch_x = np.array(preprocess(batch_x,0))
  
    with tf.GradientTape() as tape:

      
      logits = model(batch_x)
      loss_value = loss_func(batch_y, logits)

      if i % 10 == 0:
        result_loss.append(loss_value)

          #train accruacy 
        tmp = np.argmax(logits, axis=1)
        train_accuracy = np.sum(tmp==batch_y) / (batch_size*1)
        train_a.append(train_accuracy)

        #test accruacy
        #(6000/400)*10=150개씩 삽입 
        batch_tx = []
        for k in range(len(test)):
          tmp1 = test[k]
          batch_tx += tmp1[0+slice_num_test : tmp_slice_test+slice_num_test]
        slice_num_test += tmp_slice_test 
       
        test_y = np.array(label(batch_tx,1))
        test_x = np.array(preprocess(batch_tx,1))
     
        logit = model(test_x)
        tmp1 = np.argmax(logit, axis=1)
        test_accuracy = np.sum(tmp1==test_y) / int((test_len / batch_num)*10)
        test_a.append(train_accuracy)
        
        print("#",i,"th batch - train_accuracy:",train_accuracy, ", test_accuracy:",test_accuracy)

    gradients = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(i,"th batch - loss_value:",np.array(loss_value))
  
model.save("resnet18_lr4") #모델 세이브.


# 0 th batch - train_accuracy: 0.95 , test_accuracy: 0.9133333333333333
0 th batch - loss_value: 0.06804606
1 th batch - loss_value: 0.17265606
2 th batch - loss_value: 0.58567923
3 th batch - loss_value: 0.2846904
4 th batch - loss_value: 0.26656774
5 th batch - loss_value: 0.21883732
6 th batch - loss_value: 0.6599776
7 th batch - loss_value: 0.5977543
8 th batch - loss_value: 0.14606872
9 th batch - loss_value: 0.34404126
# 10 th batch - train_accuracy: 0.9166666666666666 , test_accuracy: 0.9533333333333334
10 th batch - loss_value: 0.2246713
11 th batch - loss_value: 0.14719374
12 th batch - loss_value: 0.1278471
13 th batch - loss_value: 0.14968814
14 th batch - loss_value: 0.028808402
15 th batch - loss_value: 0.60732925
16 th batch - loss_value: 0.18164387
17 th batch - loss_value: 0.32827932
18 th batch - loss_value: 0.29623508
19 th batch - loss_value: 0.13668174
# 20 th batch - train_accuracy: 0.95 , test_accuracy: 0.92
20 th batch - loss_value: 0.21575733
21 th batch - loss_

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


# 0 th batch - train_accuracy: 0.9833333333333333 , test_accuracy: 0.94
0 th batch - loss_value: 0.033890463
1 th batch - loss_value: 0.14976715
2 th batch - loss_value: 0.02431783
3 th batch - loss_value: 0.048994415
4 th batch - loss_value: 0.06447379
5 th batch - loss_value: 0.097595744
6 th batch - loss_value: 0.05155587
7 th batch - loss_value: 0.06757107
8 th batch - loss_value: 0.022503119
9 th batch - loss_value: 0.04599899
# 10 th batch - train_accuracy: 1.0 , test_accuracy: 0.96
10 th batch - loss_value: 0.03240338
11 th batch - loss_value: 0.028649164
12 th batch - loss_value: 0.026828157
13 th batch - loss_value: 0.02682708
14 th batch - loss_value: 0.0140828015
15 th batch - loss_value: 0.17302608
16 th batch - loss_value: 0.058066726
17 th batch - loss_value: 0.19963533
18 th batch - loss_value: 0.023363315
19 th batch - loss_value: 0.038717486
# 20 th batch - train_accuracy: 0.9833333333333333 , test_accuracy: 0.9533333333333334
20 th batch - loss_value: 0.052306045
21 t

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


# 0 th batch - train_accuracy: 1.0 , test_accuracy: 0.9066666666666666
0 th batch - loss_value: 0.0074306293
1 th batch - loss_value: 0.020429727
2 th batch - loss_value: 0.028384458
3 th batch - loss_value: 0.05493276
4 th batch - loss_value: 0.13640496
5 th batch - loss_value: 0.045409314
6 th batch - loss_value: 0.1897475
7 th batch - loss_value: 0.040974334
8 th batch - loss_value: 0.04402929
9 th batch - loss_value: 0.10379175
# 10 th batch - train_accuracy: 1.0 , test_accuracy: 0.96
10 th batch - loss_value: 0.04172353
11 th batch - loss_value: 0.032551102
12 th batch - loss_value: 0.0137128355
13 th batch - loss_value: 0.01895469
14 th batch - loss_value: 0.11266694
15 th batch - loss_value: 0.05304814
16 th batch - loss_value: 0.022030614
17 th batch - loss_value: 0.0374123
18 th batch - loss_value: 0.15727372
19 th batch - loss_value: 0.02152825
# 20 th batch - train_accuracy: 0.9833333333333333 , test_accuracy: 0.9333333333333333
20 th batch - loss_value: 0.02985538
21 th bat

...

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


# 0 th batch - train_accuracy: 0.9833333333333333 , test_accuracy: 0.9
0 th batch - loss_value: 0.018649485
1 th batch - loss_value: 0.019962037
2 th batch - loss_value: 0.03350825
3 th batch - loss_value: 0.07298097
4 th batch - loss_value: 0.048098866
5 th batch - loss_value: 0.18388078
6 th batch - loss_value: 0.07165577
7 th batch - loss_value: 0.088316664
8 th batch - loss_value: 0.099674836
9 th batch - loss_value: 0.030363003
# 10 th batch - train_accuracy: 0.9833333333333333 , test_accuracy: 0.9466666666666667
10 th batch - loss_value: 0.033790942
11 th batch - loss_value: 0.0073582404
12 th batch - loss_value: 0.0048233345
13 th batch - loss_value: 0.08175768
14 th batch - loss_value: 0.03598939
15 th batch - loss_value: 0.015223692
16 th batch - loss_value: 0.06785418
17 th batch - loss_value: 0.10428632
18 th batch - loss_value: 0.009165928
19 th batch - loss_value: 0.083676115
# 20 th batch - train_accuracy: 1.0 , test_accuracy: 0.92
20 th batch - loss_value: 0.0058763684
2

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


INFO:tensorflow:Assets written to: resnet18_lr4/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [ ]:
model = tf.keras.models.load_model("/content/gdrive/MyDrive/projectimg/resnet18_lr4") #모델 불러오기.

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("/content/gdrive/MyDrive/projectimg/resnet18_lr4") #tflite파일로 변환
tflite_model = converter.convert()

In [ ]:
with open('./resnet18_orii.tflite', 'wb') as f: #resnet18_orii.tflitef라는 파일이름으로 현재위치에 저장.
  f.write(tflite_model)

In [ ]:
!ls

# 새 섹션